In [2]:
import tensorflow as tf
import tensorflow_ranking as tfr
import tensorboard
from sklearn.model_selection import KFold
import numpy as np


tf.enable_eager_execution()
tf.executing_eagerly()

True

In [ ]:
data = open("/Users/apple/Downloads/libsvm_0_1000", 'r')

# # converting data into readable fomat
dataread = data.readlines()

# creating files for training and test spliting
data_train = open("/Users/apple/Downloads/libsvm15-train-1000.txt", 'w') 
data_test = open("/Users/apple/Downloads/libsvm15-test-1000.txt", 'w')

# spliting dataset into test and training

data_train_out = [] 
data_test_out = []

for line in range(0, int(len(dataread)*0.8)): 
    data_train_out.append(dataread[line]) 
for line in range(int(len(dataread)*0.8), len(dataread)): 
    data_test_out.append(dataread[line])

for item in data_train_out: 
    data_train.write(item)

for item in data_test_out: 
    data_test.write(item) 

data_train.close() 
data_test.close()

In [3]:
# Input Data

_TRAIN_DATA_PATH= "/Users/apple/Downloads/libsvm15-train-1000.txt"
_TEST_DATA_PATH= "/Users/apple/Downloads/libsvm15-test-1000.txt"


In [4]:
# Model Parameters 

_LOSS="pairwise_logistic_loss"
         
# no. of pairs

_LIST_SIZE=500 

_NUM_FEATURES=46

_BATCH_SIZE= 100 

_HIDDEN_LAYER_DIMS=["128","64"]


In [5]:
#input pipeline for tensorflow data object

def input_fn(path):
  train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
  )

  train_dataset = train_dataset.shuffle(500).repeat().batch(_BATCH_SIZE)
#   train_dataset = train_dataset.repeat().batch(_BATCH_SIZE)  
  return train_dataset.make_one_shot_iterator().get_next()

In [6]:
# Scoring function of tensorflow model

def example_feature_columns():
  """Returns the example feature columns."""
  feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
  ]
  return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
  }

def make_score_fn():
  """Returns a scoring function to build `EstimatorSpec`."""

  def _score_fn(context_features, group_features, mode, params, config):
    """Defines the network to score a documents."""
    del params
    del config
    # Define input layer.
    example_input = [
        tf.layers.flatten(group_features[name])
        for name in sorted(example_feature_columns())
    ]
    input_layer = tf.concat(example_input, 1)

    cur_layer = input_layer
    for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
      cur_layer = tf.layers.dense(
          cur_layer,
          units=layer_width,
          activation="tanh")

    logits = tf.layers.dense(cur_layer, units=1)
    
    return logits

  return _score_fn

In [7]:
# Evaluation metrics

def eval_metric_fns():
  """Returns a dict from name to metric functions.

  Returns:
    A dict mapping from metric name to a metric function with above signature.
  """
  metric_fns = {}
  metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10, 50]
  })
  
  metric_fns.update({
      "metric/%s" % name: tfr.metrics.make_ranking_metric_fn(name) for name in [
          tfr.metrics.RankingMetricKey.ARP,
          tfr.metrics.RankingMetricKey.ORDERED_PAIR_ACCURACY,
      ]
  })

  return metric_fns

In [8]:
# Building model estimators and parameters

def get_estimator(hparams):
  """Create a ranking estimator.

  Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

  Returns:
    tf.learn `Estimator`.
  """
  def _train_op_fn(loss):
    """Defines train op used in ranking head."""
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.train.get_global_step(),
        learning_rate=hparams.learning_rate,
        optimizer="Adagrad")

  ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)
  
  return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
      params=hparams)

In [9]:
# Initializing estimator

hparams = tf.contrib.training.HParams(learning_rate=0.1)
ranker = get_estimator(hparams)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/7y/q_djqpds69b8kfns_7r6xvb40000gn/T/tmps8euk12p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c3e8d8f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
# Training model

ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/7y/q_djqpds69b8kfns_7r6xvb40000gn/T/tmps8euk12p/model.ckpt.
INFO:tensorflow:loss = 0.94437134, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/7y/q_djqpds69b8kfns_7r6xvb40000gn/T/tmps8euk12p/model.ckpt.
INFO:tensorflow:Loss for final step: 0.020769034.


In [11]:
# Evaluating model on test 

ranker.evaluate(input_fn=lambda: input_fn(_TEST_DATA_PATH), steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-03T06:58:50Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/7y/q_djqpds69b8kfns_7r6xvb40000gn/T/tmps8euk12p/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2019-06-03-07:06:34
INFO:t

{'labels_mean': 0.0026331,
 'logits_mean': -5.730587,
 'loss': 0.114178896,
 'metric/arp': 17.443155,
 'metric/ndcg@1': 0.4520066,
 'metric/ndcg@10': 0.6344018,
 'metric/ndcg@3': 0.5566534,
 'metric/ndcg@5': 0.6031206,
 'metric/ndcg@50': 0.6700334,
 'metric/ordered_pair_accuracy': 0.9679511,
 'global_step': 100}

In [ ]:
# sess = tf.Session()
# file_writer = tf.summary.FileWriter('/anaconda3/pkgs/tensorboard-1.13.1-py37haf313ee_0/lib/python3.7/site-packages/tensorboard/logs', sess.graph)


In [ ]:
make_score_fn()

In [ ]:
sess = tf.Session()
with tf.Session() as sess:

    print(sess.run(logits, feed_dict=feed_dict))